# Выбор локации для скважины

Необходимо решить, где бурить новую скважину.По данным проб нефти из 3х регионов, в каждом из которых 10 000 местораждений. Для каждого месторождения представлены данные о качестве нефти и объёме запасов. Для определения региона , где добыча принесет наибольшую прибыль, построить модель машинного обучения, и оценить возмужную прибыль и риски.



## Загрузка и подготовка данных

In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState
from scipy import stats as st

In [36]:
df = pd.read_csv('/datasets/geo_data_0.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [37]:
df_1 = pd.read_csv('/datasets/geo_data_1.csv')
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [38]:
df_2 = pd.read_csv('/datasets/geo_data_2.csv')
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [39]:
df.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [40]:
df_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [41]:
df_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Провели ознакомительный осмотр представленных датасетов, прочитали файлы, вывели информацию методом info, посмотрели пару строк таблиц.

In [42]:
df.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [43]:
df_1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [44]:
df_2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [45]:
df.corr()

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


In [46]:
df_1.corr()

,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


In [47]:
df_2.corr()

,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


Познакомились с данными получше, посмотрели зависимость между параметрами, получше узнали о распределении данных, можем приступать к построению модели.

## Обучение и проверка модели

In [48]:
features = df.drop(['product','id'], axis=1)
target = df['product'] 

In [49]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345) 

In [50]:
model = LinearRegression() 
model.fit(features_train,target_train)
predictions_valid = model.predict(features_valid) 

In [51]:
result = mean_squared_error(target_valid, predictions_valid)**0.5
print("RMSE модели линейной регрессии первого региона на валидационной выборке:", result)

RMSE модели линейной регрессии первого региона на валидационной выборке: 37.5794217150813


In [52]:
features_1 = df_1.drop(['product','id'], axis=1)
target_1 = df_1['product'] 

In [53]:
features_train_1, features_valid_1, target_train_1, target_valid_1= train_test_split(
    features_1, target_1, test_size=0.25, random_state=12345) 

In [54]:
model_1 = LinearRegression() 
model_1.fit(features_train_1,target_train_1)
predictions_valid_1 = model_1.predict(features_valid_1) 

In [55]:
result_1 = mean_squared_error(target_valid_1, predictions_valid_1)**0.5
print("RMSE модели линейной регрессии второго региона на валидационной выборке:", result_1)

RMSE модели линейной регрессии второго региона на валидационной выборке: 0.893099286775616


In [56]:
features_2 = df_2.drop(['product','id'], axis=1)
target_2 = df_2['product'] 

In [57]:
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size=0.25, random_state=12345)

In [58]:
model_2 = LinearRegression() 
model_2.fit(features_train_2,target_train_2)
predictions_valid_2 = model_2.predict(features_valid_2) 

In [59]:
result_2 = mean_squared_error(target_valid_2, predictions_valid_2)**0.5
print("RMSE модели линейной регрессии третьего региона на валидационной выборке:", result_2)

RMSE модели линейной регрессии третьего региона на валидационной выборке: 40.02970873393434


Построили модели для трех регионов, по метрике *RMSE*, можно сделать вывод, что хорошая модель вышла для второго региона.

## Подготовка к расчёту прибыли

In [60]:
state = RandomState(12345)

In [61]:
coast_product = 450000
budget =  10000000000

In [62]:
volume_product = budget/(coast_product*200)
print('Срелний достаточный объем сырья для прибыльной добычи',volume_product)

Срелний достаточный объем сырья для прибыльной добычи 111.11111111111111


In [63]:
print('Средний объём сырья первого региона',df['product'].mean())
print('Средний объём сырья второго региона',df_1['product'].mean())
print('Средний объём сырья третьего региона',df_2['product'].mean())

Средний объём сырья первого региона 92.50000000000001
Средний объём сырья второго региона 68.82500000000002
Средний объём сырья третьего региона 95.00000000000004


Средний объем сырья всех трех регионов меньше вычисленного среднего достаточного объёма сырья.

## Расчёт прибыли и рисков 

In [64]:
def revenue(target, probabilities):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index].head(200)
    return (coast_product * selected.sum())-budget

In [65]:
values = []
values_1 = []
values_2 = []
series_predict = pd.Series(predictions_valid)
series_predict_1 = pd.Series(predictions_valid_1)
series_predict_2= pd.Series(predictions_valid_2)

In [66]:
target_valid = target_valid.reset_index(drop = True)
for i in range(1000):
    
    target_subsample = target_valid.sample(n=500, replace = False, random_state = state)
    pred_subsample = series_predict[target_subsample.index]
    values.append(revenue(target_subsample, pred_subsample))

In [67]:
values = pd.Series(values)
confidence_interval_mean = st.t.interval(0.95, len(values)-1, values.mean(),values.sem())
mean = values.mean()
confidence_interval = (values.quantile(0.025), values.quantile(0.975))
print("Средняя выручка первого региона:", mean)
print("Доверительный интервал выручки региона",confidence_interval)
print("95%-ый доверительный интервал средней выручки 1го региона:", confidence_interval_mean)
print('Вероятность убытков первого региона:{:.2%}'.format((values<0).mean()))

Средняя выручка первого региона: 380710890.70907485
Доверительный интервал выручки региона (-126947638.03180212, 879613967.8477958)
95%-ый доверительный интервал средней выручки 1го региона: (364624702.9511889, 396797078.4669608)
Вероятность убытков первого региона:7.20%


In [68]:
target_valid_1 = target_valid_1.reset_index(drop = True)
for i in range(1000):
    
    target_subsample_1 = target_valid_1.sample(n=500, replace = False, random_state = state)
    pred_subsample_1 = series_predict_1[target_subsample_1.index]
    values_1.append(revenue(target_subsample_1, pred_subsample_1))
values_1 = pd.Series(values_1)
confidence_interval_1 = (values_1.quantile(0.025), values_1.quantile(0.975))
confidence_interval_mean_1 = st.t.interval(0.95, len(values_1)-1, values_1.mean(),values_1.sem())
mean_1 = values_1.mean()
print("Средняя выручка второго региона:", mean_1)
print("Доверительный интервал выручки второго региона",confidence_interval_1)
print("95%-ый доверительный интервал средней выручки 2го региона:", confidence_interval_mean_1)
print('Вероятность убытков второго региона:{:.2%}'.format((values_1<0).mean()))

Средняя выручка второго региона: 454785434.76571304
Доверительный интервал выручки второго региона (46730084.76920634, 840213356.26002)
95%-ый доверительный интервал средней выручки 2го региона: (441979241.37927634, 467591628.15214974)
Вероятность убытков второго региона:1.30%


In [69]:
target_valid_2 = target_valid_2.reset_index(drop = True)
for i in range(1000):
    
    target_subsample_2 = target_valid_2.sample(n=500, replace = False, random_state = state)
    pred_subsample_2 = series_predict_2[target_subsample_2.index]
    values_2.append(revenue(target_subsample_2, pred_subsample_2))
values_2 = pd.Series(values_2)
confidence_interval_2 = (values_2.quantile(0.025), values_2.quantile(0.975))
confidence_interval_mean_2 = st.t.interval(0.95, len(values_2)-1, values_2.mean(),values_2.sem())
mean_2 = values_2.mean()
print("Средняя выручка третьего региона:", mean_2)
print("Доверительный интервал выручки третьего региона",confidence_interval_2)
print("95%-ый доверительный интервал средней выручки 3го региона:", confidence_interval_2)
print('Вероятность убытков третьего региона:{:.2%}'.format((values_2<0).mean()))

Средняя выручка третьего региона: 389217073.68557006
Доверительный интервал выручки третьего региона (-115609565.77767344, 906512590.1730094)
95%-ый доверительный интервал средней выручки 3го региона: (-115609565.77767344, 906512590.1730094)
Вероятность убытков третьего региона:7.30%


Проведя анализ средней выручки, и убытков, можно сделать вывод, что скважины необходимо брать во втором регионе, во втором регионе по расчитаным прогнозам нас ожидает большая выручка и меньшая вероятность убытков, идеальное сочетание.

## Вывод

Для полученя результатов была проделана работа :
- проведен анализ представленных данных, на их взаимосвязь и познакомлись с типами и форматами данных,
- разбили выборки на обучающаю и валидационную(в нашем случае она и тестовая),
- построили модели для каждого региона  и оценили модель при помощи метрики *RMSE*, точнее всех оказалась модель второго региона,
- произвели оценочный расчет средних объемов запасов, для достижения требуемой прибыли,
- процедурой bootstrap оценили выручку с каждого региона и доверительный интервал, а также вероятность убытков, при разработке скважин в конкретном регионе.

В ходе работы можно сделать вывод о доходности разработки скважин во втором регионе, выручка выше, а вероятность убытков ниже, чем в двух других представленных к оценки регионах.